In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/muchmoredata/MuchMoreData.csv


Ce dataset se compose d’abstracts de publications scientifiques médicales, qui traitent de différents domaines: Cardiologie, Ophtalmologie etc. On a donc un problème de classification multiclasses (assigne une seule des classes possibles à un document). La catégorie à laquelle appartient un document correspond à la première partie de son nom.

A partir de ce dataset, nous allo

*    

Proposer une analyse exploratoire des éon
*    ées
Présenter et commenter quelques visualisations et métriques pertine*    ntes
Comparer les performances de deux approches pour la classification des doc :  
1.   SVM l vs e
2.   Naiv*    e Bayes
Proposer une exploration interactive des données et résultats

# Analyse exploratoire des données

# Quelques visualisations et métriques pertinentes

# Comparaisons des performances de SVM linéaire et Naive Bayes

# Exploration interactive des données et résultats

# Conclusions